In [ ]:
import os, sys
sys.path.append('../')
sys.path.append('../../SICGAN/')
sys.path.append('../../meshrcnn/')
import argparse
import random 
import numpy as np
from tqdm import tqdm
import torch
import re, yaml, pickle
from torch import nn, optim
from torch.autograd import Variable
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.io import load_obj, save_obj
from pytorch3d.loss import mesh_laplacian_smoothing, mesh_edge_loss
from style_transfer.config import Config
from style_transfer.models.base_nn import GraphConvClf
from style_transfer.config import Config
from style_transfer.utils.torch_utils import train_val_split, save_checkpoint, accuracy
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation #import FuncAnimation
from matplotlib.animation import FuncAnimation
from collections import OrderedDict
from ico_objects import ico_disk
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objects as go
init_notebook_mode(connected=True)

In [ ]:
def plot_pointcloud(mesh, title=""):
    # Sample points uniformly from the surface of the mesh.
    points = sample_points_from_meshes(mesh, 5000)
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)    
    fig = plt.figure(figsize=(5, 5))
    ax = Axes3D(fig)
    ax.scatter3D(x, z, -y)
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.savefig(title+'.png')

In [ ]:
from PyGEL3D import gel
from PyGEL3D import js
def plot_mesh(mesh):
    save_obj('mesh.obj', mesh.verts_packed(), mesh.faces_packed())
    js.set_export_mode()
    m = gel.obj_load('mesh.obj')
    js.display(m, smooth=False)

In [ ]:
def plot_mesh_plt(mesh, title=""):
    # Sample points uniformly from the surface of the mesh.
    verts = mesh.verts_packed().cpu()
    faces = mesh.faces_packed().cpu()
    x = verts[:,0]
    y = verts[:,1]
    z = verts[:,2]
    i = faces[:,0]
    j = faces[:,1]
    k = faces[:,2]
    fig = plt.figure(figsize=(5, 5))
    ax = Axes3D(fig)
    ax.plot_trisurf(x, y, z, triangles = [i,j,k])
    ax.set_title(title)
    ax.view_init(120, 25)
    plt.show()
    
def gif_pointcloud(mesh, title=""):
    # Sample points uniformly from the surface of the mesh.
    points = sample_points_from_meshes(mesh, 5000)
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)    
    fig = plt.figure(figsize=(5, 5))
    ax = Axes3D(fig)
    ax.scatter3D(x, z, -y)
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    ax.set_title(title)
    def update(i):
        ax.view_init(190,i)
    anim = FuncAnimation(fig, update, frames=np.arange(0, 100), interval=200)

In [ ]:
def load_mesh(tst_obj):
    verts, faces, aux = load_obj(tst_obj)
    mesh = Meshes(verts=[verts], faces=[faces.verts_idx]).cuda()
    return mesh

In [ ]:
def plot_mesh_plotly(mesh):
    verts = mesh.verts_packed().cpu()
    faces = mesh.faces_packed().cpu()
    fig = go.Figure(data=[
        go.Mesh3d(
            opacity = 0.5,
            flatshading=True,
            x = verts[:,0],
            y = verts[:,1],
            z = verts[:,2],
            i = faces[:,0],
            j = faces[:,1],
            k = faces[:,2]
        )
    ])
    fig.show()

In [ ]:
table_pth = '/scratch/jiadeng_root/jiadeng/shared_data/datasets/ShapeNetCore.v1/04379243/1028a9cbaa7a333230bbd4cddd04c77b/model.obj'
airplane_pth = '/scratch/jiadeng_root/jiadeng/shared_data/datasets/ShapeNetCore.v1/02691156/105f7f51e4140ee4b6b87e72ead132ed/model.obj'
chair_pth = '/scratch/jiadeng_root/jiadeng/shared_data/datasets/ShapeNetCore.v1/03001627/1016f4debe988507589aae130c1f06fb/model.obj'
rifle_pth = '/scratch/jiadeng_root/jiadeng/shared_data/datasets/ShapeNetCore.v1/04090263/10439f1f81fbee202be79d8b285c1e//model.obj'
table, airplane, chair, rifle = load_mesh(table_pth), load_mesh(airplane_pth), load_mesh(chair_pth), load_mesh(rifle_pth)
disk = ico_disk(level=0, device=device)

In [ ]:
from shapenet.modeling.mesh_arch import Pixel2MeshHead
from shapenet.config import Config as pix_Config

In [ ]:
pix_C = pix_Config('../../meshrcnn/configs/shapenet/pixel2mesh_R50.yaml', [])

In [ ]:
a = torch.load('pixel2mesh_R50.pth')["best_states"]["model"]
sd = OrderedDict([(k.replace('module.',''), v) for k, v in a.items()])

In [ ]:
G = Pixel2MeshHead(pix_C).cuda()
G.load_state_dict(sd)

In [ ]:
G.mesh_head

In [ ]:
device = torch.device("cuda:0")
#device = torch.device("cpu")
#
## load in trained graph convolutional NN classification model
graphconv_model_path = '../results/model/exp_03_17_00_51_59_10classes'
#content_mesh_path = args.filename_obj
idx_best_loss=16
#LAYER_ID = 1
#NODE_ID = 100
ITERS = 400  
MESH_LR =1.0

cfg = Config('../config/style_transfer_train.yml')
#
## SET UP model and optimizer
classification_model = GraphConvClf(cfg).cuda()
classification_model.load_state_dict(torch.load(graphconv_model_path+'/model@epoch'+str(idx_best_loss)+'.pkl')['state_dict'])
for pp in classification_model.parameters():
    pp.requires_grad=False

In [ ]:
trg_features =  classification_model.get_forward_feats(mesh=airplane, layername='gconv2')

In [ ]:
def gconv_loss(verts, edges, classification_model, layer_name='gonv0', feat=None, method='deepdream', target=airplane):
    ii = int(layer_name[-1])
    if ii != 0: 
        i = ii-1
        verts = classification_model.get_forward_feats(mesh=None,verts=verts, edges=edges ,layername='gconv'+str(i))
            
    center_verts = verts[0]
    neighbor_verts = verts[1:9]
    center_verts = classification_model.gconvs[ii].w0(center_verts)
    neighbor_verts = classification_model.gconvs[ii].w1(neighbor_verts)
    ##
    nnfeatures = center_verts + torch.sum(neighbor_verts, dim=0)
    if method=='feat_inv':
        trg_features =  classification_model.get_forward_feats(mesh=target, layername=layer_name)
#         trg_feat = torch.mean(trg_features, dim=0).cuda()
        trg_feat = trg_features[trg_features.norm(dim=0).argmax(), :]
        loss = torch.mean((nnfeatures - trg_feat)**2)
    elif method=='deepdream':
        loss = nnfeatures[feat].norm()
    return loss

In [ ]:
src_mesh = ico_disk(level=1).cuda()
plot_mesh_plt(src_mesh)
verts = src_mesh.verts_packed()
faces = src_mesh.faces_packed()
edges = src_mesh.edges_packed()
which_layer = 'gconv0'
target = airplane
method='deepdream'
which_feature = 1
# offset_verts = torch.full(verts.shape, 0.0, device=device, requires_grad=True)
verts = Variable(verts, requires_grad=True)
classification_model.zero_grad()
initial_lr = 3.0
decayed_lr = 0.1
decay_lr_at = 100
# optimizer = torch.optim.Adam([offset_verts], lr=initial_lr)

In [ ]:
 for which_feature in range(1):
        for i in range(10):
            new_verts = verts
            loss = gconv_loss(new_verts, edges, classification_model, which_layer, which_feature, method, target)
#             if i%50==0:
            mode_mesh = Meshes(verts=[new_verts], faces=[faces])
#             loss += 1*mesh_edge_loss(mode_mesh)
            print(loss.item())
            loss.backward()
            verts.data -= 1e-9*verts.grad.data
    #     m = Meshes([verts.detach()], [faces])
    #     plot_mesh_plt(m)
    #     plt.show()

In [ ]:
m = Meshes([verts.detach()], [faces])
plot_mesh_plt(m)

In [ ]:
plot_mesh(m)